In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import random
from copy import deepcopy
import numpy as np
from datetime import datetime, timedelta


dir = os.path.split(os.getcwd())[0]
if dir not in sys.path:
    sys.path.append(dir)

from model import Course
from controller import search_classes
from view import Schedule, print_schedule
from useful_functions import schedule_generator, valid_schedule, time_of_day_score
from hillclimbing import energy_function

In [ ]:
example_schedule = [
    "CS2810",
    "CS2510",
    "CS2511",
    "ENGW1111",
    "MKTG2201"
]

courses = []
for course in example_schedule:
    banner_courses = search_classes(course[:-4], course[-4:])

    assert len(banner_courses) > 0, "Course doesn't exist or isn't in the spring 2024 semester"
    
    courses.append(banner_courses)

In [ ]:
for course in courses[0]:
    if course.is_async():
        print("ASYNC")

In [ ]:
random_schedule = schedule_generator(courses)

In [ ]:
def hillclimb(schedule, tod_pref, day_off_pref, time_betweem_pref, iterations, all_courses):
    energy__time = []
    # variables to hold the best results 
    best_solution = deepcopy(schedule)
    best_energy = energy_function(schedule, tod_pref, day_off_pref, time_betweem_pref)
    num_classes = len(schedule)
    
    for _ in range(iterations):
        #a random class index in the schedule and num of possible courses for that class
        class_idx = random.randint(0, num_classes-1)
        num_courses = len(all_courses[class_idx])
        
        # copy of the current best maze
        new_schedule = deepcopy(best_solution)

        # upating the random class to be a random course form the 2d list of all courses
        new_schedule[class_idx] = all_courses[class_idx][random.randint(0,num_courses-1)]

        # prevents the tile change from creating an unsolvable maze
        while not valid_schedule(new_schedule):
            new_schedule[class_idx] = all_courses[class_idx][random.randint(0,num_courses-1)]

        # finds the energy value of the new maze
        new_energy = energy_function(new_schedule, tod_pref, day_off_pref, time_betweem_pref)
        # compares the new energy value to the previous best
        if  new_energy > best_energy:
            best_solution = deepcopy(new_schedule)
            best_energy = new_energy

        energy__time.append(best_energy)



    return best_solution, best_energy, energy__time

        
def hillclimb_random_restarts(schedule, tod_pref, day_off_pref, time_betweem_pref, iterations, num_restarts, all_courses):
    # variables to hold the best results 
    best_solution = deepcopy(schedule)

    best_energy = energy_function(schedule, tod_pref, day_off_pref, time_betweem_pref)
    energy__time_restart = []
    for _ in range(num_restarts):
        new_solution, new_energy, energy__time = hillclimb(schedule, tod_pref, day_off_pref, time_betweem_pref, iterations, all_courses)

        energy__time_restart.append(energy__time)
        if new_energy > best_energy:
            best_solution = deepcopy(new_solution)
            best_energy = new_energy
            best_energy__time = energy__time

    return best_solution, best_energy, energy__time_restart, best_energy__time


In [ ]:
time_preference = "m"
day_off = "fri"
time_between = 10


# best_solution, best_energy, energy__time = hillclimb(random_schedule, time_preference, day_off, time_between, 1000, courses)
best_solution, best_energy, energy__time_restarts, best_energy__time = hillclimb_random_restarts(random_schedule, time_preference, day_off, time_between, 100, 20, courses)

In [ ]:
# 
for hill_climb_data in energy__time_restarts:
    plt.plot(hill_climb_data)

# plt.plot(energy__time)
plt.xlabel('Iteration')
plt.ylabel('Energy')
plt.title('Best Energy vs. Iteration for each Restart')
plt.legend([f'Restart {i+1}' for i in range(len(energy__time_restarts))], loc='upper center', bbox_to_anchor=(1.2, 1.05))
plt.savefig("./data/all_energies.png", bbox_inches="tight")

plt.show()



In [ ]:
plt.plot(best_energy__time)
plt.xlabel('Iteration')
plt.ylabel('Energy')
plt.title('Best Energy vs. Iteration')


plt.savefig("./data/best_energy.png")
plt.show()

In [ ]:
# how start time impacts time of day score
new_points_for_tods = {
    "m": [],
    "a": [],
    "e": []
}

old_points_for_tods = {
    "m": [],
    "a": [],
    "e": []
}

def tod_match(startTime, tod_pref):
    return (tod_pref == "m" and int(startTime) < 1200) or \
        (tod_pref == "a" and int(startTime) >= 1200 and int(startTime) <= 1600) or \
        (tod_pref == "e" and int(startTime) > 1600)
       

            
fig, ax = plt.subplots()
x_ticks = []
for tod_pref, new_points_for_tod in new_points_for_tods.items():
    x_ticks = []
    start_datetime = datetime(2023, 1, 1, 0, 0, 0)
    end_datetime = start_datetime + timedelta(hours=24)
    hour_interval = timedelta(minutes=15)

    # Loop through all datetimes in the 24-hour period
    current_datetime = start_datetime
    while current_datetime < end_datetime:
        x_ticks.append(current_datetime)
        new_points_for_tod.append(time_of_day_score(current_datetime, tod_pref))
        current_datetime += hour_interval
    
    ax.plot(new_points_for_tod, label=tod_pref)


ax.set_xlabel('Time Of Day')
ax.set_ylabel('Points')
ax.set_xticks(np.arange(0, len(x_ticks), 10), labels=[x_ticks[i].strftime("%H:%M") for i in range(0, len(x_ticks), 10)], rotation=70)

ax.set_title('Points Given for Time of Day')
ax.legend()

plt.savefig("./data/new_tod.png", bbox_inches="tight")
plt.show()
plt.close()


fig, ax = plt.subplots()
x_ticks = []
for tod_pref, old_points_for_tod in old_points_for_tods.items():
    x_ticks = []

    start_datetime = datetime(2023, 1, 1, 0, 0, 0)
    end_datetime = start_datetime + timedelta(hours=24)
    hour_interval = timedelta(minutes=15)

    # Loop through all datetimes in the 24-hour period
    current_datetime = start_datetime
    while current_datetime < end_datetime:
        x_ticks.append(current_datetime)
        if tod_match(current_datetime.strftime("%H%M"), tod_pref):
            old_points_for_tod.append(15)
        else:
            old_points_for_tod.append(0)
        current_datetime += hour_interval

    ax.plot(old_points_for_tod, label=tod_pref)

ax.set_xlabel('Time Of Day')

ax.set_ylabel('Points')

ax.set_xticks(np.arange(0, len(x_ticks), 10), labels=[x_ticks[i].strftime("%H:%M") for i in range(0, len(x_ticks), 10)], rotation=70)
# ax.set_xticks()
print([y_ticks[i] for i in range(0, len(y_ticks), 5)])
ax.set_title('Points Given for Time of Day')
ax.legend()
plt.savefig("./data/old_tod.png", bbox_inches="tight")
plt.show()
plt.close()


In [ ]:
best_schedule = Schedule(best_solution)
print_schedule(best_schedule)

In [ ]:
start_schedule = Schedule(random_schedule)
print_schedule(start_schedule)